In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

import random
import ast
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

In [96]:
raw_recipes = pd.read_csv('food.csv')

#Cleaning and EDA

In [97]:
raw_recipes.head(2)

,RecipeID,recipe_name,recipies,By,Description,RequiredTime,Servings,Ingredients,Directions,Nutritions,Tags,ImageLink,ExtraTags
0,1,Aamabtvaran recipe - How to make Aamabtvaran,Aamabtvaran,madhuri_a1,Aamabtvaran is a maharashtrian way of cooking ...,40,4,1 cup toovar (arhar) dal\n1/2 tsp turmeric pow...,Method\nPressure cook the dal with asafoeitda ...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Maharashtrian Dal, Varan / Amti / Kalvan']",https://www.tarladalal.com/images/recipe/addne...,"['Maharashtrian Dal, Varan / Amti / Kalvan', '..."
1,2,"Aloo and Kand Rasawala Shaak, Faraal Recipe re...","Aloo and Kand Rasawala Shaak, Faraal Recipe",Tarla Dalal,This recipe is a mouth-watering preparation of...,24,4,1 cup boiled potato cubes\n1 cup steamed purpl...,"Method\nCombine the curds, rajgira flour, salt...","{""energy"" : ""157 cal"", ""protein"" : ""2.8 g"", ""c...","['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara...",https://www.tarladalal.com/members/9306/big/bi...,"['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara..."


In [98]:
raw_recipes.shape

(436, 13)

In [99]:
raw_recipes.describe()

,RecipeID,RequiredTime,Servings
count,436.000000,436.000000,436.000000
mean,218.500000,24.635321,5.011468
std,126.006614,13.650061,12.523352
min,1.000000,0.000000,1.000000
25%,109.750000,15.000000,2.000000
50%,218.500000,25.000000,4.000000
75%,327.250000,35.000000,4.000000
max,436.000000,55.000000,250.000000


In [100]:
raw_recipes.describe(include='object')

,recipe_name,recipies,By,Description,Ingredients,Directions,Nutritions,Tags,ImageLink,ExtraTags
count,436,436,436,436,436,436,436,436,436,436
unique,434,434,103,435,430,431,204,371,329,435
top,Spring Dosa ( Mumbai Roadside Recipes) recipe ...,Spring Dosa ( Mumbai Roadside Recipes),Tarla Dalal,What does spring mean to you? A bounty of vegg...,"1 1/2 cups soaked sago (sabudana) , refer hand...","Method\n\nWash, drain and soak the sabudana in...","{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...",['Maharashtrian Sweet Dishes'],https://www.tarladalal.com/images/recipe/addne...,"['South Indian', 'Different Types of Dosa', 'S..."
freq,2,2,288,2,2,2,231,9,107,2


In [101]:
raw_recipes.isnull().sum()

RecipeID        0
recipe_name     0
recipies        0
By              0
Description     0
RequiredTime    0
Servings        0
Ingredients     0
Directions      0
Nutritions      0
Tags            0
ImageLink       0
ExtraTags       0
dtype: int64

In [102]:
raw_recipes.fillna(0)

,RecipeID,recipe_name,recipies,By,Description,RequiredTime,Servings,Ingredients,Directions,Nutritions,Tags,ImageLink,ExtraTags
0,1,Aamabtvaran recipe - How to make Aamabtvaran,Aamabtvaran,madhuri_a1,Aamabtvaran is a maharashtrian way of cooking ...,40,4,1 cup toovar (arhar) dal\n1/2 tsp turmeric pow...,Method\nPressure cook the dal with asafoeitda ...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Maharashtrian Dal, Varan / Amti / Kalvan']",https://www.tarladalal.com/images/recipe/addne...,"['Maharashtrian Dal, Varan / Amti / Kalvan', '..."
1,2,"Aloo and Kand Rasawala Shaak, Faraal Recipe re...","Aloo and Kand Rasawala Shaak, Faraal Recipe",Tarla Dalal,This recipe is a mouth-watering preparation of...,24,4,1 cup boiled potato cubes\n1 cup steamed purpl...,"Method\nCombine the curds, rajgira flour, salt...","{""energy"" : ""157 cal"", ""protein"" : ""2.8 g"", ""c...","['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara...",https://www.tarladalal.com/members/9306/big/bi...,"['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara..."
2,3,Aloo Chaat ( Mumbai Roadside Recipes ) recipe ...,Aloo Chaat ( Mumbai Roadside Recipes ),Tarla Dalal,aloo chaat recipe | Mumbai roadside aloo chaat...,14,2,For The Aloo Chaat Marinated Potatoes\n1 1/4 c...,Method\nFor the aloo chaat marinated potatoes\...,"{""energy"" : ""656 cal"", ""protein"" : ""17.8 g"", ""...","['Indian Veg Recipes', 'Mumbai Street Food', '...",https://www.tarladalal.com/members/9306/big/bi...,"['Indian Veg Recipes', 'Mumbai Street Food', '..."
3,4,Aloo ki Kadhi Recipe recipe - How to make Aloo...,Aloo ki Kadhi Recipe,vrunda naik,Seeking to satiate your appetite on fasting da...,40,4,"For The Pakodas\n1/2 kg potatoes , boiled , pe...",Method\nFor the pakodas\nTo make pakodas for t...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Faral, Faraal', 'Maharashtrian Upva...",https://www.tarladalal.com/images/recipe/addne...,"['Gujarati Faral, Faraal', 'Maharashtrian Upva..."
4,5,Alu Bhat recipe - How to make Alu Bhat,Alu Bhat,arathi_receipe,A vegetable dish which is common in most benga...,40,6,To Be Blended Into A Smooth Paste\n1/2 cup cho...,Method\nHeat the oil in a kadai and add cloves...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Khichdi', 'Maharashtrian Bhaat (Ric...",https://www.tarladalal.com/images/recipe/addne...,"['Gujarati Khichdi', 'Maharashtrian Bhaat (Ric..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,432,Whole Wheat Roti Chivda recipe - How to make W...,Whole Wheat Roti Chivda,Tarla Dalal,Whole Wheat Roti Chivda is a light and tasty s...,25,3,For The Whole Wheat Rotis\n1 1/4 cups whole wh...,Method\nFor the whole wheat rotis\nCombine the...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Maharashtrian Snacks Nashta', 'Maharashtrian...",https://www.tarladalal.com/members/9306/big/bi...,"['Maharashtrian Snacks Nashta', 'Maharashtrian..."
432,433,"Yam Raita, Farali Suran Raita Recipe recipe - ...","Yam Raita, Farali Suran Raita Recipe",Tarla Dalal,yam raita recipe | farali suran raita | senai ...,10,1,"For Yam Raita\n1 cup peeled , boiled and mashe...","Method\nFor yam raita\nTo make yam raita, comb...","{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Kachumber / Chutneys / Achar', 'Guj...",https://www.tarladalal.com/members/9306/big/bi...,"['Gujarati Kachumber / Chutneys / Achar', 'Guj..."
433,434,Yummy Pudina Rice recipe - How to make Yummy P...,Yummy Pudina Rice,Jyoti V B,"This is a tasty rice, try this with tomato and...",30,4,2 cups long grained rice (basmati)\nsalt to ta...,Method\nPressure cook the rice till 2 whistles...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...",['Maharashtrian Bhaat (Rice)'],https://www.tarladalal.com/members/575789/big/...,"['Maharashtrian Bhaat (Rice)', 'Yummy Pudina R..."
434,435,Zunka recipe - How to make Zunka,Zunka,Tarla Dalal,zunka recipe | jhunka | Marathi zunka bhakar |...,29,4,For Zunka\n1 cup 

In [103]:
raw_recipes.dropna()

,RecipeID,recipe_name,recipies,By,Description,RequiredTime,Servings,Ingredients,Directions,Nutritions,Tags,ImageLink,ExtraTags
0,1,Aamabtvaran recipe - How to make Aamabtvaran,Aamabtvaran,madhuri_a1,Aamabtvaran is a maharashtrian way of cooking ...,40,4,1 cup toovar (arhar) dal\n1/2 tsp turmeric pow...,Method\nPressure cook the dal with asafoeitda ...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Maharashtrian Dal, Varan / Amti / Kalvan']",https://www.tarladalal.com/images/recipe/addne...,"['Maharashtrian Dal, Varan / Amti / Kalvan', '..."
1,2,"Aloo and Kand Rasawala Shaak, Faraal Recipe re...","Aloo and Kand Rasawala Shaak, Faraal Recipe",Tarla Dalal,This recipe is a mouth-watering preparation of...,24,4,1 cup boiled potato cubes\n1 cup steamed purpl...,"Method\nCombine the curds, rajgira flour, salt...","{""energy"" : ""157 cal"", ""protein"" : ""2.8 g"", ""c...","['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara...",https://www.tarladalal.com/members/9306/big/bi...,"['Gujarati Shaak Sabzi', 'Gujarati Faral, Fara..."
2,3,Aloo Chaat ( Mumbai Roadside Recipes ) recipe ...,Aloo Chaat ( Mumbai Roadside Recipes ),Tarla Dalal,aloo chaat recipe | Mumbai roadside aloo chaat...,14,2,For The Aloo Chaat Marinated Potatoes\n1 1/4 c...,Method\nFor the aloo chaat marinated potatoes\...,"{""energy"" : ""656 cal"", ""protein"" : ""17.8 g"", ""...","['Indian Veg Recipes', 'Mumbai Street Food', '...",https://www.tarladalal.com/members/9306/big/bi...,"['Indian Veg Recipes', 'Mumbai Street Food', '..."
3,4,Aloo ki Kadhi Recipe recipe - How to make Aloo...,Aloo ki Kadhi Recipe,vrunda naik,Seeking to satiate your appetite on fasting da...,40,4,"For The Pakodas\n1/2 kg potatoes , boiled , pe...",Method\nFor the pakodas\nTo make pakodas for t...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Faral, Faraal', 'Maharashtrian Upva...",https://www.tarladalal.com/images/recipe/addne...,"['Gujarati Faral, Faraal', 'Maharashtrian Upva..."
4,5,Alu Bhat recipe - How to make Alu Bhat,Alu Bhat,arathi_receipe,A vegetable dish which is common in most benga...,40,6,To Be Blended Into A Smooth Paste\n1/2 cup cho...,Method\nHeat the oil in a kadai and add cloves...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Khichdi', 'Maharashtrian Bhaat (Ric...",https://www.tarladalal.com/images/recipe/addne...,"['Gujarati Khichdi', 'Maharashtrian Bhaat (Ric..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,432,Whole Wheat Roti Chivda recipe - How to make W...,Whole Wheat Roti Chivda,Tarla Dalal,Whole Wheat Roti Chivda is a light and tasty s...,25,3,For The Whole Wheat Rotis\n1 1/4 cups whole wh...,Method\nFor the whole wheat rotis\nCombine the...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Maharashtrian Snacks Nashta', 'Maharashtrian...",https://www.tarladalal.com/members/9306/big/bi...,"['Maharashtrian Snacks Nashta', 'Maharashtrian..."
432,433,"Yam Raita, Farali Suran Raita Recipe recipe - ...","Yam Raita, Farali Suran Raita Recipe",Tarla Dalal,yam raita recipe | farali suran raita | senai ...,10,1,"For Yam Raita\n1 cup peeled , boiled and mashe...","Method\nFor yam raita\nTo make yam raita, comb...","{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...","['Gujarati Kachumber / Chutneys / Achar', 'Guj...",https://www.tarladalal.com/members/9306/big/bi...,"['Gujarati Kachumber / Chutneys / Achar', 'Guj..."
433,434,Yummy Pudina Rice recipe - How to make Yummy P...,Yummy Pudina Rice,Jyoti V B,"This is a tasty rice, try this with tomato and...",30,4,2 cups long grained rice (basmati)\nsalt to ta...,Method\nPressure cook the rice till 2 whistles...,"{""energy"" : ""N/A"", ""protein"" : ""N/A"", ""carbs"" ...",['Maharashtrian Bhaat (Rice)'],https://www.tarladalal.com/members/575789/big/...,"['Maharashtrian Bhaat (Rice)', 'Yummy Pudina R..."
434,435,Zunka recipe - How to make Zunka,Zunka,Tarla Dalal,zunka recipe | jhunka | Marathi zunka bhakar |...,29,4,For Zunka\n1 cup 

In [104]:
raw_recipes.duplicated().sum()

0

In [105]:
raw_recipes.columns

Index(['RecipeID', 'recipe_name', 'recipies', 'By', 'Description',
       'RequiredTime', 'Servings', 'Ingredients', 'Directions', 'Nutritions',
       'Tags', 'ImageLink', 'ExtraTags'],
      dtype='object')

#Recomandation systemm


In [106]:
new_recipes = raw_recipes[['RecipeID','recipies','RequiredTime','Ingredients','Description']]

In [107]:
new_recipes.shape

(436, 5)

In [108]:

new_recipes.to_csv('new_recipes.csv',index = False)

In [109]:
new_recipes.columns

Index(['RecipeID', 'recipies', 'RequiredTime', 'Ingredients', 'Description'], dtype='object')

In [110]:
new_recipes.rename(columns={'recipies': 'title'}, inplace=True)

C:\Users\nidhi\AppData\Local\Temp\ipykernel_16452\2030411897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipes.rename(columns={'recipies': 'title'}, inplace=True)


In [111]:
new_recipes.head(2)

,RecipeID,title,RequiredTime,Ingredients,Description
0,1,Aamabtvaran,40,1 cup toovar (arhar) dal\n1/2 tsp turmeric pow...,Aamabtvaran is a maharashtrian way of cooking ...
1,2,"Aloo and Kand Rasawala Shaak, Faraal Recipe",24,1 cup boiled potato cubes\n1 cup steamed purpl...,This recipe is a mouth-watering preparation of...


In [112]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features=436 , stop_words = 'english')

In [113]:
cv

CountVectorizer(max_features=436, stop_words='english')

In [114]:
vector = cv.fit_transform(new_recipes['title'].values.astype('U')).toarray()

In [115]:
#vector = cv.fit_transform(new_recipes['recipies'].values.astype('U')).toarray()

In [116]:
vector.shape

(436, 436)

In [117]:
from sklearn.metrics.pairwise import cosine_similarity

In [118]:
similarity = cosine_similarity(vector)

In [119]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.18257419, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.18257419, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5       ,
        1.        ]])

In [120]:
new_recipes [new_recipes['title'] == "Zunka"].index[0]

434

In [121]:
distance = sorted(list(enumerate(similarity[434])) , reverse = True , key = lambda vector:vector[1])
for i in distance[0:2]:
    print(f"Recipe Name: {new_recipes.iloc[i[0]].title}")
    print(f"RequiredTime: {new_recipes.iloc[i[0]].RequiredTime}")
    print(f"Ingredients: {new_recipes.iloc[i[0]].Ingredients}")
    print(f"Description: {new_recipes.iloc[i[0]].Description}")
    print("-------------------------------------------------------------------------------------------------------------------")


Recipe Name: Zunka
RequiredTime: 29
Ingredients: For Zunka
1 cup besan (bengal gram flour)
2 tbsp oil
1/2 tsp mustard seeds ( rai / sarson)
1/2 tsp cumin seeds (jeera)
1/4 tsp asafoetida (hing)
1 tsp chopped ginger (adrak)
2 tbsp chopped green chillies
2 tbsp chopped garlic (lehsun)
1 cup chopped onions
1/4 tsp turmeric powder (haldi)
salt to taste
1/2 cup chopped coriander (dhania)

For The Tempering
1 tbsp oil
1/2 tsp mustard seeds ( rai / sarson)
1/4 tsp asafoetida (hing)
8 to 10 curry leaves (kadi patta)
2 tsp chopped garlic (lehsun)
2 dry red chillies (pandi)

For Serving
rice (chawal) bhakri
Description: zunka recipe | jhunka | Marathi zunka bhakar | Maharashtrian zunka |

zunka is an authentic Maharashtrian delicacy which is slightly spicy but very satisfying in texture, flavour and aroma. Learn how to make Marathi zunka bhakar.

Traditional Maharashtrian and spicy, the Maharashtrian zunka is considered by many as a dry version of the famous pitla. It is like a curry of ginger, 

In [122]:
new_recipes.columns

Index(['RecipeID', 'title', 'RequiredTime', 'Ingredients', 'Description'], dtype='object')

In [123]:
def recommand(raw_recipes):
  index = new_recipes [new_recipes['title'] == raw_recipes].index[0]
  distance = sorted(list(enumerate(similarity[index])) , reverse = True , key = lambda vector:vector[1])
  for i in distance[0:5]:
    print(f"Recipe Name: {new_recipes.iloc[i[0]].title}")
    print(f"RequiredTime: {new_recipes.iloc[i[0]].RequiredTime}")
    print(f"Ingredients: {new_recipes.iloc[i[0]].Ingredients}")
    print(f"Description: {new_recipes.iloc[i[0]].Description}")
    print("-------------------------------------------------------------------------------------------------------------------")

In [124]:
recommand(input("Enter the rcipe name: ") )

#Baitha Varan
#Zunka
#Alu Bhat
# Amrit Modak
#Batata Tomato Rassa Bhaji
#Bhakarwadi(4)

Recipe Name: Baitha Varan
RequiredTime: 30
Ingredients: 3/4 cup toovar (arhar) dal
1/4 tsp turmeric powder (haldi)
salt to taste
1/4 tsp asafoetida (hing)
1/2 tsp roasted cumin seeds (jeera) powder
1 1/2 cups boiled mixed vegetables (carrot , cauliflower , potatoes , onions etc.)
1/2 cup green paste
2 tsp sugar
2 tsp lemon juice
1/4 cup finely chopped coriander (dhania) for the garnish
Description: The green paste gives this otherwise simple dish a complete makeover! It increases the vitamin A content, while the vegetables add fibre. Vitamin A increases immunity and also nourishes our skin and improves vision. Despite the interesting flavour and health benefits, this dish is surprisingly easy to make!
-------------------------------------------------------------------------------------------------------------------
Recipe Name: Methiche Varan
RequiredTime: 35
Ingredients: 1 cup finely chopped fenugreek (methi) leaves
1 cup toovar (arhar) dal , washed and drained
1/4 tsp turmeric powder

In [125]:
import pickle
pickle.dump(new_recipes,open('recipes_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.load(open('recipes_list.pkl','rb'))


,RecipeID,title,RequiredTime,Ingredients,Description
0,1,Aamabtvaran,40,1 cup toovar (arhar) dal\n1/2 tsp turmeric pow...,Aamabtvaran is a maharashtrian way of cooking ...
1,2,"Aloo and Kand Rasawala Shaak, Faraal Recipe",24,1 cup boiled potato cubes\n1 cup steamed purpl...,This recipe is a mouth-watering preparation of...
2,3,Aloo Chaat ( Mumbai Roadside Recipes ),14,For The Aloo Chaat Marinated Potatoes\n1 1/4 c...,aloo chaat recipe | Mumbai roadside aloo chaat...
3,4,Aloo ki Kadhi Recipe,40,"For The Pakodas\n1/2 kg potatoes , boiled , pe...",Seeking to satiate your appetite on fasting da...
4,5,Alu Bhat,40,To Be Blended Into A Smooth Paste\n1/2 cup cho...,A vegetable dish which is common in most benga...
...,...,...,...,...,...
431,432,Whole Wheat Roti Chivda,25,For The Whole Wheat Rotis\n1 1/4 cups whole wh...,Whole Wheat Roti Chivda is a light and tasty s...
432,433,"Yam Raita, Farali Suran Raita Recipe",10,"For Yam Raita\n1 cup peeled , boiled and mashe...",yam raita recipe | farali suran raita | senai ...
433,434,Yummy Pudina Rice,30,2 cups long grained rice (basmati)\nsalt to ta...,"This is a tasty rice, try this with tomato and..."
434,435,Zunka,29,For Zunka\n1 cup besan (bengal gram flour)\n2 ...,zunka recipe | jhunka | Marathi zunka bhakar |...
